In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import os
print(os.listdir("../input"))
import lightgbm as lgb
import gc
import featuretools as ft
es = ft.EntitySet()

In [ ]:
path = "../input/elo-merchant-category-recommendation/"

In [ ]:
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
ID=test['card_id']
ytrain  = train['target']
xtrain = train.drop('target',axis=1)
alldata = pd.concat([test,xtrain])

In [ ]:
#merchants = pd.read_csv(path+'merchants.csv')
#historical_transactions = pd.read_csv(path+'historical_transactions.csv')
#new_merchant_transactions = pd.read_csv(path+'new_merchant_transactions.csv')

In [ ]:
#减少数据集占用的内存
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
 #number_feature
qnum = ['card_id','installments','purchase_amount','numerical_1', 'numerical_2','avg_sales_lag3', 'avg_purchases_lag3',
        'avg_sales_lag6', 'avg_purchases_lag6','avg_sales_lag12','avg_purchases_lag12']

In [ ]:
 def fm_num(qnum)  :
    merchants = pd.read_csv(path+'merchants.csv')
    historical_transactions = pd.read_csv(path+'new_merchant_transactions.csv')
#    historical_transactions = pd.concat([historical_transactions,pd.read_csv(path+'new_merchant_transactions.csv')],axis=1)
    historical_transactions.drop(['category_1','category_2','merchant_category_id'],axis=1,inplace=True)
    merchants.drop(['city_id','state_id','subsector_id'],axis=1,inplace=True)
    historical_transactions = reduce_mem_usage(historical_transactions)
    merchants = reduce_mem_usage(merchants)
    historical_transactions = pd.merge(historical_transactions,merchants,on='merchant_id',how='left')
    historical_transactions = reduce_mem_usage(historical_transactions)

    x1=historical_transactions[qnum]
    
    del historical_transactions
    del merchants
    print(gc.collect())
    
    #开始使用featuretools
    import featuretools as ft
    es = ft.EntitySet()
    es = es.entity_from_dataframe(entity_id="traindf",#设定聚合id
                                dataframe=alldata,#加载主数据集
                                index="card_id",
                       #         time_index="first_active_month",
                                  #Set Categorica Featrue# 设定类别特征，不设定默认为数值特征
                                variable_types={"feature_1": ft.variable_types.Categorical,
                                               "feature_2": ft.variable_types.Categorical,
                                               "feature_3": ft.variable_types.Categorical}
                                 )

    x1.reset_index(inplace=True)
    es = es.entity_from_dataframe(entity_id="1df",
                                dataframe=x1,#加载需合并的数据集，如有
                       #         time_index="first_active_month",
                                 )
    new_relationship = ft.Relationship(es["traindf"]["card_id"],#设定数据集之间关联字段，相当于merge
                                       es["1df"]["card_id"])
    es = es.add_relationship(new_relationship)

    del x1
    print(gc.collect())

    feature_matrix,feature_defs = ft.dfs(entityset=es,
                                          target_entity="traindf",
                                          verbose=True,
                                    #      n_jobs=2
                                         )
    feature_matrix = reduce_mem_usage(feature_matrix)
    es = ft.EntitySet()
    print(gc.collect())
    return feature_matrix

In [ ]:
feature_matrix1 = fm_num(qnum)
feature_matrix1=feature_matrix1.reset_index()
feature_matrix1.to_csv('new_num.csv',index=None)

qnum = ['installments','purchase_amount','numerical_1', 'numerical_2','avg_sales_lag3', 'avg_purchases_lag3',
        'avg_sales_lag6', 'avg_purchases_lag6','avg_sales_lag12','avg_purchases_lag12']

def fm_cat(qnum)  :
    merchants = pd.read_csv(path+'merchants.csv')
    historical_transactions = pd.read_csv(path+'new_merchant_transactions.csv')
   # historical_transactions = pd.concat([historical_transactions,pd.read_csv(path+'new_merchant_transactions.csv')],axis=1)
    historical_transactions.drop(['category_1','category_2','merchant_category_id'],axis=1,inplace=True)
    merchants.drop(['city_id','state_id','subsector_id'],axis=1,inplace=True)
    historical_transactions = reduce_mem_usage(historical_transactions)
    merchants = reduce_mem_usage(merchants)
    historical_transactions = pd.merge(historical_transactions,merchants,on='merchant_id',how='left')
    historical_transactions = reduce_mem_usage(historical_transactions)
    
    x1=historical_transactions.drop(qnum,axis=1)
    
    del historical_transactions
    del merchants
    print(gc.collect())
    import featuretools as ft
    es = ft.EntitySet()    
    
    #Set Categorica Featrue
    variable_types={"merchant_id": ft.variable_types.Categorical,
        "merchant_group_id": ft.variable_types.Categorical,
       "merchant_category_id": ft.variable_types.Categorical,
       "subsector_id": ft.variable_types.Categorical,
       "category_1": ft.variable_types.Categorical,
       "most_recent_purchases_range": ft.variable_types.Categorical,
       "most_recent_sales_range": ft.variable_types.Categorical,
       "active_months_lag3": ft.variable_types.Categorical,
       "active_months_lag6": ft.variable_types.Categorical,
       "active_months_lag12": ft.variable_types.Categorical,
       "category_4": ft.variable_types.Categorical,
       "city_id": ft.variable_types.Categorical,
       "state_id": ft.variable_types.Categorical,
       "category_2": ft.variable_types.Categorical,
        'category_3': ft.variable_types.Categorical,
         "month_lag": ft.variable_types.Categorical,}
    

    es = es.entity_from_dataframe(entity_id="traindf",
                                dataframe=alldata,
                                index="card_id",
                       #         time_index="first_active_month",
                                variable_types={"feature_1": ft.variable_types.Categorical,
                                               "feature_2": ft.variable_types.Categorical,
                                               "feature_3": ft.variable_types.Categorical}
                                 )

    x1.reset_index(inplace=True)
    es = es.entity_from_dataframe(entity_id="1df",
                                dataframe=x1,
                                time_index='purchase_date',
                                variable_types=variable_types
                                 )
    new_relationship = ft.Relationship(es["traindf"]["card_id"],
                                       es["1df"]["card_id"])
    es = es.add_relationship(new_relationship)

    del x1
    print(gc.collect())

    feature_matrix,feature_defs = ft.dfs(entityset=es,
                                          target_entity="traindf",
                                          verbose=True,
                                   #       n_jobs=2
                                         )
    feature_matrix = reduce_mem_usage(feature_matrix)
    es = ft.EntitySet()
    print(gc.collect())
    return feature_matrix

feature_matrix2 = fm_cat(qnum)
feature_matrix2=feature_matrix2.reset_index()
feature_matrix2.to_csv('new_cat.csv',index=None)